# Project pRF analysis results to pycortex surfaces
Make sure you have created the pycortex entries for these individuals (https://github.com/VisionandCognition/NHP-pycortex)     
Strategy:    
- Load all (unthresholded) results into sorted numpy arrays with nibabel
- Also create pycortex volume objects
- Save them in an ordered dictionary
- Define some functions for masking
- Perform any additional post-processing

### Import the required modules

In [1]:
import cortex
import nibabel as nib 
import numpy as np
import pandas as pd
import os, shutil, copy

from matplotlib import pyplot as plt

### Set subject name and path to FitResults

In [2]:
#subj = 'Danny' # 'Danny'/'Eddy'
subs = ['Danny', 'Eddy']
FitResPath = os.path.join('/Users','chris','Documents','MRI_ANALYSIS','NHP-PRF','FitResults')
ManualMaskPath = os.path.join('/Users','chris','Dropbox','GIT_Support','NHP-BIDS','manual-masks')

### Specify which models and results to include
Making changes here may or may not break subsequent as it relies on some of these things being present (e.g., R2, R2_1, and R2_2)

In [3]:
Models = {
    'dhrf' : ['linhrf_cv1_dhrf','linhrf_cv1_dhrf_neggain','csshrf_cv1_dhrf','doghrf_cv1_dhrf'],
    'mhrf' : ['linhrf_cv1_mhrf','linhrf_cv1_mhrf_neggain','csshrf_cv1_mhrf','doghrf_cv1_mhrf'],
    'names' : ['lin','lin_ng','css','dog'],
}
Res_type = ['ANG', 'ECC', 'EXPT', 'FWHM', 'GAIN', 'IMAG', 'REAL', 'RFS', 'X', 'Y', 'NAMP', 'SDRATIO']
xfm = 'epi2surf'

### Create a volume to vertex mapper for this subject

In [4]:
Vol2Vert={}
for s in subs:
    Vol2Vert[s] = cortex.get_mapper(s, xfm, 'line_nearest', recache=True)

### Create a dictionary to collect all FitResults in

In [5]:
# this will be the structure of data collection
SR={}
for s in subs:
    SR[s] = {
        'subject' : s,
        'xfm' : xfm,
        'mHRF' : {
            'arr' : {},
            'vol' : {},
        },
        'dHRF' : {
            'arr' : {},
            'vol' : {},
        },
    }

### Populate the dictionary with the data

In [6]:
# Get the data
for s in subs:
    for h in ['m','d']:
        print('Processing ' + h + 'HRF:', end='')
        for M in Models[h + 'hrf']:
            # get model idx so we create a shorter variable name
            midx = Models[h + 'hrf'].index(M)
            print(' ' + Models['names'][midx], end='')

            # get the mean R2 map
            volpath = os.path.join(FitResPath,'MRI',s.lower(),M,'Sess-' + M + '_meanR2.nii.gz')
            volpath1 = os.path.join(FitResPath,'MRI',s.lower(),M,'Sess-' + M + '_R2_1.nii.gz')
            volpath2 = os.path.join(FitResPath,'MRI',s.lower(),M,'Sess-' + M + '_R2_2.nii.gz')

            # load the results into numpy arrays with nibabel
            R2 = np.array(nib.load(volpath).dataobj)
            R2_1 = np.array(nib.load(volpath1).dataobj)
            R2_2 = np.array(nib.load(volpath2).dataobj)

            # convert to pycortex volumes
            R2v = cortex.Volume(R2.transpose(2,1,0), s, xfm)
            R2_1v = cortex.Volume(R2_1.transpose(2,1,0), s, xfm)
            R2_2v = cortex.Volume(R2_2.transpose(2,1,0), s, xfm)

            # add info to dictionaries
            # numpy arrays
            FitRes = {
                'R2' : R2,
                'R2_1' : R2_1,
                'R2_2' : R2_2
                }
            # pycortex volumes
            FitRes_vol = {
                'R2' : R2v,
                'R2_1' : R2_1v,
                'R2_2' : R2_2v
                }

            # also get othere results
            for R in Res_type:
                volpath = os.path.join(FitResPath,'MRI',s.lower(),M,'TH_0', R + '_th0.nii.gz')
                if os.path.exists(volpath):
                    FitRes[R] = np.array(nib.load(volpath).dataobj)
                    FitRes_vol[R] = cortex.Volume(FitRes[R].transpose(2,1,0), s, xfm)

            # bring it all together
            SR[s][h + 'HRF']['arr'][Models['names'][midx]] = FitRes
            SR[s][h + 'HRF']['vol'][Models['names'][midx]] = FitRes_vol
        print('')

Processing mHRF: lin lin_ng css dog
Processing dHRF: lin lin_ng css dog
Processing mHRF: lin lin_ng css dog
Processing dHRF: lin lin_ng css dog


In [7]:
if False:
    subj = 'Eddy'
    # Check in webviewer 
    cortex.webgl.show(data=SR[subj]['mHRF']['vol']['lin'])
    cortex.webgl.show(data=SR[subj]['mHRF']['vol']['lin_ng'])

In [8]:
R2_diffarray={}
for s in subs:
    R2_diffarray[s] = SR[s]['mHRF']['arr']['lin']['R2'] - SR[s]['mHRF']['arr']['lin_ng']['R2']
    SR[s]['mHRF']['vol']['lin_ng']['R2_DIFF'] = cortex.Volume(R2_diffarray[s].transpose(2,1,0), s, xfm)
    cortex.webgl.show(data=SR[s]['mHRF']['vol']['lin_ng'])

Started server on port 62500
Started server on port 18588


### Do some checks to see if this worked as expected
Once we know it worked we can switch this off again

In [9]:
if False:
    subj='Danny'
    # Inspect the result volumes (should be numpy arrays)
    RR = copy.copy(SR[subj]['mHRF']['arr']['lin']['R2']) # copy the R2 values for some model
    RR[RR < 5] ='nan' # threshold it to some level
    
    cortex.webgl.show(data=cortex.Volume(RR.transpose(2,1,0), subj, xfm)) # check to seeit worked

### Define a function to mask the results based on R2 values
This function makes a deepcopy of the input dictionary to prevend overwriting it with masked data. Than masks it with the provided R2-threshold

In [10]:
def r2mask(DataDict,RTH):
    DD = copy.deepcopy(DataDict) # copy the original data so it won't get overwritten
    # mask the numpy arrays by inserting nan's
    for hrf in ['mHRF','dHRF']:
        for m in DD[hrf]['arr']:
            # mask all available outputs except for R2
            for res in DD[hrf]['arr'][m]:
                if res is not 'R2':
                    with np.errstate(invalid='ignore'):
                        DD[hrf]['arr'][m][res][ DD[hrf]['arr'][m]['R2'] < RTH ] = 'nan'
                    # also convert to the pycortex volume
                    DD[hrf]['vol'][m][res] = cortex.Volume(DD[hrf]['arr'][m][res].transpose(2,1,0), DD['subject'], DD['xfm'])
            # mask R2            
            with np.errstate(invalid='ignore'):
                DD[hrf]['arr'][m]['R2'][ DD[hrf]['arr'][m]['R2'] < RTH ] = 'nan' 
            DD[hrf]['vol'][m]['R2m'] = cortex.Volume(DD[hrf]['arr'][m]['R2'].transpose(2,1,0), DD['subject'], DD['xfm'])

    return DD

Check whether the masking function works

In [11]:
if False:
    mFR = r2mask(SR[subj],2)
    cortex.webgl.show(data=mFR['mHRF']['vol']['lin'])

### Get D99 atlas information 

In [12]:
# read atlas labels
D99_inFunc = {}
for subj in subs:
    D99_inFunc[subj]={}
    D99_inFunc[subj]['path'] = os.path.join(ManualMaskPath,'sub-' + subj.lower(),'atlas','D99_in_' + subj + '_adj_inFunc.nii')
    D99_inFunc[subj]['arr'] = np.array(nib.load(D99_inFunc[subj]['path']).dataobj)
    D99_inFunc[subj]['vol'] = cortex.Volume(D99_inFunc[subj]['arr'].transpose(2,1,0), subj, xfm)
    D99_inFunc[subj]['labelpath'] = os.path.join(ManualMaskPath,'sub-' + subj.lower(),'atlas','D99_labeltable_reformat.txt')

    D99_inFunc[subj]['labels'] = {}
    with open(D99_inFunc[subj]['labelpath']) as f:
        for line in f:
            labelnum, label = line.strip().split(' ',1)
            D99_inFunc[subj]['labels'][label.strip()] = int(labelnum)
    #print(D99_inFunc['labels'])

In [13]:
if False:
    subj='Danny'
    cortex.webgl.show(data=D99_inFunc[subj]['vol'])

### Define a function that returns ROI names from voxel label number

In [14]:
def get_roiname(LabelDict,LabelNum):
    for name, number in LabelDict.items():
        if number == LabelNum:
            return name

In [15]:
# test if the get_roiname function works
if False:
    subj='Danny'
    roi = get_roiname(D99_inFunc[subj]['labels'],147)
    print(roi)

### Define a function that returns data masked by ROI

In [16]:
def roimask(DataDict,AtlasDict,rois):
    # DataDict is the dictionary of results
    # AtlasDict is a dictionary of D99 atlas info
    # rois is a list of rois to include
    
    DD = copy.deepcopy(DataDict) # copy the original data so it won't get overwritten
    AA = copy.deepcopy(AtlasDict) # copy the original data so it won't get overwritten
    
    # Create a mask that includes all rois in the list
    mask = np.zeros(AA['arr'].shape)
    for r in rois:
        tmask = AA['arr']==AA['labels'][r]
        mask = mask + tmask
    mask[mask > 0]
    
    # mask the numpy arrays by inserting nan's
    for hrf in ['mHRF','dHRF']:
        for m in DD[hrf]['arr']:
            # mask all available outputs except for R2
            for res in DD[hrf]['arr'][m]:
                with np.errstate(invalid='ignore'):
                    DD[hrf]['arr'][m][res][ mask < 1 ] = 'nan'
                # also convert to the pycortex volume
                DD[hrf]['vol'][m][res] = cortex.Volume(DD[hrf]['arr'][m][res].transpose(2,1,0), DD['subject'], DD['xfm'])
    return DD

In [17]:
# test this function
if False:
    subj='Danny'
    V1Res = roimask(SR[subj],D99_inFunc[subj],['V1'])
    cortex.webgl.show(data=V1Res['mHRF']['vol']['lin'])

In [18]:
# test masking by ROI and R2
if False:
    subj='Danny'
    V1Res = roimask(SR[subj],D99_inFunc[subj],['V1'])
    mV1Res = r2mask(V1Res,4)
    cortex.webgl.show(data=mV1Res['mHRF']['vol']['lin'])

## Create visualizations for manuscript figures

In [19]:
subj = 'Eddy'
hrf = 'mHRF'
model = 'css'

M1 = r2mask(SR[subj],5)
#cortex.webgl.show(data=M1[hrf]['vol'][model])

## Inspect the localization of negative model benefits

In [20]:
TH = 5
hrf = 'mHRF'

DoG={}; PLIN={}; ULIN={}
   
# NB! Rememeber that masks are 'inverse masks' here, used to set the non-wanted voxels to 'nan'    
    
    
for s in subs:  
    DoG[s]={}
    PLIN[s]={}
    ULIN[s]={}
    
    DoG[s]['R2'] = copy.deepcopy(SR[s][hrf]['arr']['dog']['R2'])
    PLIN[s]['R2'] = copy.deepcopy(SR[s][hrf]['arr']['lin']['R2'])
    ULIN[s]['R2'] = copy.deepcopy(SR[s][hrf]['arr']['lin_ng']['R2'])

    ULIN[s]['R2_above_th'] = copy.copy(ULIN[s]['R2'])
    ULIN[s]['R2_sub_th'] = copy.copy(ULIN[s]['R2'])
    ULIN[s]['THMASK'] = ULIN[s]['R2'] > TH
    ULIN[s]['R2_above_th'][ np.invert(ULIN[s]['THMASK']) ] = 'nan'
    ULIN[s]['R2_sub_th'][ ULIN[s]['THMASK'] ] = 'nan'
    
    PLIN[s]['R2_above_th'] = copy.copy(PLIN[s]['R2'])
    PLIN[s]['R2_sub_th'] = copy.copy(PLIN[s]['R2'])
    PLIN[s]['THMASK'] = PLIN[s]['R2'] > TH
    PLIN[s]['R2_above_th'][ np.invert(PLIN[s]['THMASK']) ] = 'nan'
    PLIN[s]['R2_sub_th'][ PLIN[s]['THMASK'] ] = 'nan'

    DoG[s]['R2_above_th'] = copy.copy(DoG[s]['R2'])
    DoG[s]['R2_sub_th'] = copy.copy(DoG[s]['R2'])
    DoG[s]['THMASK'] = DoG[s]['R2'] > TH
    DoG[s]['R2_above_th'][ np.invert(DoG[s]['THMASK']) ] = 'nan'
    DoG[s]['R2_sub_th'][ DoG[s]['THMASK'] ] = 'nan'

## Inspect U-LIN gain and DoG nAMP

In [21]:
UD={}; DN={};
for s in subs:
    UD[s]={}
    UD[s]['Gain'] = SR[s][hrf]['arr']['lin_ng']['GAIN']
    UD[s]['ULIN_R2'] = ULIN[s]['R2_above_th']
    UD[s]['Gain_masked'] = copy.copy(UD[s]['Gain'])
    UD[s]['Gain_masked'][np.invert(ULIN[s]['THMASK'])] = 'nan'
    
    UD[s]['nAmp'] = SR[s][hrf]['arr']['dog']['NAMP']
    UD[s]['DoG_R2'] = DoG[s]['R2_above_th']
    UD[s]['nAmp_masked'] = copy.copy(UD[s]['nAmp'])
    UD[s]['nAmp_masked'][np.invert(DoG[s]['THMASK'])] = 'nan'  
    
    
    for a in UD[s]:
        print(a)
        DN[s + '_' + a] = cortex.Volume(UD[s][a].transpose(2,1,0), s, xfm)
        
cortex.webgl.show(data=DN)

Gain
ULIN_R2
Gain_masked
nAmp
DoG_R2
nAmp_masked
Gain
ULIN_R2
Gain_masked
nAmp
DoG_R2
nAmp_masked
Started server on port 59404


<JS: window.viewer>

## U-LIN vs P-LIN    
Here we check the voxels for which both U-LIN and P-LIN fit decently

In [21]:
# P-LIN & U-LIN R2 > TH
# Map dR2 U-LIN minus P-LIN
UP={}; DN={};
for s in subs:
    UP[s]={}
    UP[s]['dR2_th'] = ULIN[s]['R2_above_th'] - PLIN[s]['R2_above_th']
    
    UP[s]['nanMASK'] = np.isnan(UP[s]['dR2_th'])
    UP[s]['Ecc'] = SR[s][hrf]['arr']['lin']['ECC'] 
    UP[s]['Ecc_masked'] = copy.copy(UP[s]['Ecc'])
    UP[s]['Ecc_masked'][UP[s]['nanMASK']] = 'nan'
    
    for a in UP[s]:
        print(a)
        DN[s + '_' + a] = cortex.Volume(UP[s][a].transpose(2,1,0), s, xfm)
        
cortex.webgl.show(data=DN)

dR2_th
nanMASK
Ecc
Ecc_masked
dR2_th
nanMASK
Ecc
Ecc_masked
Started server on port 63297


<JS: window.viewer>

## U-LIN vs P-LIN     
Here we check the voxels for which U-LIN performs well, but P-LIN does not

In [22]:
# U-LIN R2 > TH & P-LIN < TH
# Map dR2 U-LIN minus P-LIN
UP={}; DN={};
for s in subs:
    UP[s]={}
    UP[s]['dR2_th'] = ULIN[s]['R2_above_th'] - PLIN[s]['R2_sub_th']
    #UP[s]['ULIN-MASK'] = np.invert( ULIN[s]['THMASK'] )
    #UP[s]['PLIN-MASK'] = PLIN[s]['THMASK']    
    UP[s]['nanMASK'] = np.isnan(UP[s]['dR2_th'])
    
    UP[s]['Ecc'] = SR[s][hrf]['arr']['lin']['ECC'] 
    UP[s]['Ecc_masked'] = copy.copy(UP[s]['Ecc'])
    UP[s]['Ecc_masked'][UP[s]['nanMASK']] = 'nan'

    UP[s]['R2_ULIN'] = ULIN[s]['R2']
    UP[s]['R2_PLIN'] = PLIN[s]['R2']
    
    for a in UP[s]:
        print(a)
        DN[s + '_' + a] = cortex.Volume(UP[s][a].transpose(2,1,0), s, xfm)
        
cortex.webgl.show(data=DN)

dR2_th
nanMASK
Ecc
Ecc_masked
R2_ULIN
R2_PLIN
dR2_th
nanMASK
Ecc
Ecc_masked
R2_ULIN
R2_PLIN
Started server on port 62742


<JS: window.viewer>

Stopping server
Stopping server


## DoG vs P-LIN    
Here we check the voxels for which DoG outperforms P-LIN, while both perform ok.

In [23]:
# DoG R2 > TH & P-LIN > TH
# Map dR2 DoG minus P-LIN
UP={}; DN={};
for s in subs:
    UP[s]={}
    UP[s]['dR2_th'] = DoG[s]['R2_above_th'] - PLIN[s]['R2_above_th'] 
    UP[s]['nanMASK'] = np.isnan(UP[s]['dR2_th'])
    
    UP[s]['Ecc'] = SR[s][hrf]['arr']['lin']['ECC'] 
    UP[s]['Ecc_masked'] = copy.copy(UP[s]['Ecc'])
    UP[s]['Ecc_masked'][UP[s]['nanMASK']] = 'nan'
        
    UP[s]['R2_DoG'] = DoG[s]['R2']
    UP[s]['R2_PLIN'] = PLIN[s]['R2']
    
    for a in UP[s]:
        print(a)
        DN[s + '_' + a] = cortex.Volume(UP[s][a].transpose(2,1,0), s, xfm)
        
cortex.webgl.show(data=DN)

dR2_th
nanMASK
Ecc
Ecc_masked
R2_DoG
R2_PLIN
dR2_th
nanMASK
Ecc
Ecc_masked
R2_DoG
R2_PLIN
Started server on port 3447


<JS: window.viewer>

Stopping server
Stopping server
Stopping server
